### Homework Nr. 7
#### Tasks:
✅ Complete week 3 and 4 of the fifth course from specialization<br/>
✅ Watch whole [playlist](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) from 3blue1brown, where he explains math behind neural networks and how you can visualize it.<br/>
✅ Implement  a GPT model from scratch, while watching the [video](https://youtu.be/kCc8FmEb1nY?si=ZrDbF9MUeCpt9Lto) from Andrej Karpathy, a former director of AI at Tesla <br/>


# Building GPT (Generatively Pretrained Transformer)

In [2]:
# get the tiny shakespeare dataset
with open("input.txt", "r") as f:
    text = f.read()

In [3]:
# print out the first 250 characters
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
#  unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print("vocab size:", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [5]:
# create a mapping from unique characters to indices
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


In [6]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [9]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"When input is {context} the target is {target}")

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences we process in parallel?
block_size = 8  # what is the maximum cointext length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(
        len(data) - block_size, (batch_size,)
    )  # starting index for each sequence
    x = torch.stack([data[i : i + block_size] for i in ix])  # we index the data tensor
    y = torch.stack(
        [data[i + 1 : i + block_size + 1] for i in ix]
    )  # y is the same but one ahead
    return x, y


xb, yb = get_batch("train")
print("inputs:", xb, xb.shape)
print("targets:", yb, yb.shape)
print("-------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target is {target}")

inputs: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) torch.Size([4, 8])
targets: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]]) torch.Size([4, 8])
-------------
When input is [24] the target is 43
When input is [24, 43] the target is 58
When input is [24, 43, 58] the target is 5
When input is [24, 43, 58, 5] the target is 57
When input is [24, 43, 58, 5, 57] the target is 1
When input is [24, 43, 58, 5, 57, 1] the target is 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
When input is [44] the target is 53
When input is [44, 53] the target is 56
When input is [44, 53, 56] the target is 1
When input is [44, 53, 56, 1] the target is 58
When input is [44, 53, 56, 1, 58] the

In [12]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


### Bigram language model
it predicts the next token based only on the current token. The key idea behind this model is that it learns a direct mapping from a given token to a probability distribution over the vocabulary for the next token.



In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from the lookup table
        #  each token maps directly to a one-hot vector.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) shaped
        logits = self.token_embedding_table(idx)
        # (Batch = 4, Time = block = 8, Channels = vocab_size = 65)
        if targets is None:
            loss = None
        else:
            # To make it compatible with cross-entropy loss, we need to reshape it to (B*T, C)
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)  # or -1 instead of B*T
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) shaped
        for _ in range(max_new_tokens):
            logits, loss = self(idx)  # calling forward

            logits = logits[:, -1, :]  # keep only the last token for each sequence

            probs = F.softmax(
                logits, dim=-1
            )  # apply softmax to get the probabilities (B, C)
            idx_next = torch.multinomial(
                probs, num_samples=1
            )  # get the max likely next token (B, 1= num_samples)
            idx = torch.cat([idx, idx_next], dim=1)  # append next to the sequence
        return idx


model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros(
    (1, 1), dtype=torch.long
)  # we start with an empty sequence (0 is newline)
next = model.generate(idx, max_new_tokens=100)  # generate 100 new tokens
print(decode(next[0].tolist()))  # convert to list and decode

torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [14]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for step in range(10000):

    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5589075088500977


In [16]:
idx = torch.zeros(
    (1, 1), dtype=torch.long
)  # we start with an empty sequence (0 is newline)
next = model.generate(idx, max_new_tokens=300)  # generate 100 new tokens
print(decode(next[0].tolist()))  # convert to list and decode


Ong h hasbe pave pirance
RDe hicomyonthar's
PES:
AKEd ith henourzincenonthioneir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprroutherccnohathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so itJas
Waketancotha:
h hay.JUCLUKn prids, r loncave w hollular s O:
HIs; ht anjx


##### The math trick in self-attention

In [17]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# bag of words - bow
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [19]:
x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

more efficient matrix multiplication

In [20]:
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a =")
print(a)
print("--")
print("b =")
print(b)
print("--")
print("c =")
print(c)

a =
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c =
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


2+6+6 = 14 <br/>
7+4+5 = 16

In [21]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [22]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a =")
print(a)
print("--")
print("b =")
print(b)
print("--")
print("c =")
print(c)

a =
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c =
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


100 * 266 = 2
110 * 266 = 2+6 = 8
111 * 266 = 2+6+6 = 14

In [23]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("a =")
print(a)
print("--")
print("b =")
print(b)
print("--")
print("c =")
print(c)

a =
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b =
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c =
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


2+6 /2 = 4.0 <br/>
2+6+6 / 3 = 4.6 <br/>
average

#### Back to work


In [28]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [29]:
# bag of words - bow
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [31]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x  # (B, T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version3 use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)  # dim is row
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [ ]:
# version 4: self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei = q @ k.transpose(
    -2, -1
)  # (B,T,16)@(B,16,T)-->(B,T,T) transpose the last 2 dim T and H


tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))

# this line if decoder, delete this line if encoder
wei = wei.masked_fill(
    tril == 0, float("-inf")
)  # delete this line if we want all the nodes to communicate(the previous nodes, and next nodes)
wei = F.softmax(wei, dim=-1)  # dim is row

v = value(x)  # (B, T, 16)
out = wei @ v  # (B, T, T) @ (B, T, 16) ---> (B, T, 16)
out.shape

torch.Size([4, 8, 16])

In [43]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Different since each batch has different words/chars